In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44588
2,Huelva,Confirmados PDIA 14 días,953
3,Huelva,Tasa PDIA 14 días,"185,7084396983456"
4,Huelva,Confirmados PDIA 7 días,353
5,Huelva,Tasa PDIA 7 dias,"68,78812089560964"
6,Huelva,Total Confirmados,44788
7,Huelva,Curados,40202
8,Huelva,Fallecidos,405


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44588.0


/tmp/ipykernel_4735/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12369.0


/tmp/ipykernel_4735/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4735/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4735/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4735/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1776 personas en los últimos 7 días 

Un positivo PCR cada 669 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44588.0,353.0,953.0,513170.0,68.788121,185.708440,54.0
Huelva-Costa,26344.0,219.0,589.0,289548.0,75.635128,203.420504,38.0
Condado-Campiña,13883.0,100.0,265.0,156231.0,64.007783,169.620626,13.0
Cartaya,1870.0,25.0,80.0,20083.0,124.483394,398.346861,9.0
Huelva (capital),12369.0,81.0,215.0,143837.0,56.313744,149.474753,7.0
Ayamonte,1998.0,21.0,49.0,21104.0,99.507202,232.183472,6.0
Palos de la Frontera,999.0,12.0,16.0,11742.0,102.197241,136.262988,6.0
Lepe,2958.0,22.0,57.0,27880.0,78.909613,204.447633,4.0
Isla Cristina,3056.0,29.0,75.0,21393.0,135.558360,350.581966,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,24.0,6.0,7.0,614.0,977.198697,1140.065147,3.0
Cañaveral de León,30.0,1.0,4.0,396.0,252.525253,1010.101010,0.0
Sanlúcar de Guadiana,34.0,2.0,3.0,403.0,496.277916,744.416873,0.0
Rosal de la Frontera,71.0,1.0,12.0,1697.0,58.927519,707.130230,0.0
Cabezas Rubias,52.0,1.0,4.0,706.0,141.643059,566.572238,0.0
San Bartolomé de la Torre,375.0,12.0,21.0,3761.0,319.064079,558.362138,1.0
Escacena del Campo,231.0,3.0,12.0,2287.0,131.176213,524.704854,0.0
Higuera de la Sierra,62.0,4.0,6.0,1291.0,309.837335,464.756003,0.0
Cerro de Andévalo (El),195.0,4.0,10.0,2327.0,171.895144,429.737860,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rosal de la Frontera,71.0,1.0,12.0,1697.0,58.927519,707.130230,0.0,0.083333
Beas,356.0,1.0,9.0,4341.0,23.036167,207.325501,0.0,0.111111
San Juan del Puerto,752.0,2.0,15.0,9411.0,21.251727,159.387950,0.0,0.133333
Villanueva de los Castillejos,206.0,1.0,6.0,2825.0,35.398230,212.389381,0.0,0.166667
Valverde del Camino,830.0,2.0,11.0,12750.0,15.686275,86.274510,1.0,0.181818
Bollullos Par del Condado,1257.0,1.0,5.0,14387.0,6.950719,34.753597,0.0,0.200000
Aracena,664.0,3.0,15.0,8255.0,36.341611,181.708056,1.0,0.200000
Lucena del Puerto,362.0,3.0,14.0,3261.0,91.996320,429.316161,0.0,0.214286
Bonares,452.0,2.0,9.0,6060.0,33.003300,148.514851,1.0,0.222222
